In [4]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul 30 12:24:38 2023

@author: yswav
"""

import pandas as pd
import tulipy as ti
import numpy as np

df = pd.read_excel("HINDALCO_1D.xlsx")
open = df['open']
high = df['high']
low = df['low']
close = df['close']


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [9]:
df['st'], df['s_upt'], df['st_dt'] = get_supertrend(high, low, close, lookback=10, multiplier=3)

C:\Users\yswav\AppData\Local\Temp\ipykernel_18552\246321578.py:15: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  final_bands.iloc[:,1] = final_bands.iloc[:,0]


KeyError: 0

In [ ]:

# Calculate the Average True Range (ATR) using tulipy
atr_period = 14  # You can adjust this value as per your preference
atr = ti.atr(df['high'].values, df['low'].values, df['close'].values, atr_period)

In [ ]:
df = df.drop(index=df.index[:len(df) - len(atr)])
df['atr'] = atr
print(df)

In [ ]:

# Calculate the Weekly Volume-Weighted Average Price (VWAP)
# Assuming you have a column 'vwap_weekly' in your DataFrame representing the weekly VWAP
# If not, you can calculate it using the following:
# df['vwap_weekly'] = df['close'] * df['volume'] / df['volume'].rolling(window=5).sum()
# # You can adjust the window size (5 in this case) to calculate the VWAP as per your requirement


In [ ]:
def vwap(df):
    q = df.volume.values
    p = df.close.values
    return df.assign(vwap=(p * q).cumsum() / q.cumsum())

df = vwap(df)

In [ ]:
df

In [ ]:

# Define your buy and sell conditions
buy_condition = (df['close'] > df['atr']) & (df['close'] > df['vwap_weekly'])
sell_condition = (df['close'] < df['atr']) & (df['close'] < df['vwap_weekly'])


In [ ]:
df

In [ ]:
df[['close', 'vwap_weekly','atr']].plot()

In [ ]:

# Initialize a new column to hold trading signals
df['signal'] = 0

# Apply the strategy based on the conditions
df.loc[buy_condition, 'signal'] = 1  # Buy signal
df.loc[sell_condition, 'signal'] = -1  # Sell signal

# Backtest the strategy (assuming you have a 'position' column representing the current position)
# If the 'signal' is 1, buy the instrument
# If the 'signal' is -1, sell the instrument
# If the 'signal' is 0, do nothing

df['position'] = df['signal'].diff().fillna(0)

# Calculate the strategy returns
df['strategy_returns'] = df['position'] * df['close'].pct_change().fillna(0)

# Calculate the cumulative returns
df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

# Plot the cumulative returns to visualize the performance of the strategy
df['cumulative_returns'].plot(figsize=(10, 6))